In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

Définitions des fonctions statistiques et graphiques des caractéristiques

In [ ]:
df = pd.read_csv("/workspaces/datasciencetest_reco_plante/notebooks/Plant_V_Seg_all_features.csv")

In [ ]:
df.columns

tableau identifiant les chevauchements avec IQR

In [ ]:
# Génération d'un tableau identifiant les chevauchements avec IQR

def iqr_overlap_table(df, features, label_col="disease_label", healthy_label="healthy"):
    """
    Renvoie un DataFrame listant, pour chaque feature,
    les maladies dont l'IQR chevauche celui de 'healthy'.
    """
    results = []
    diseases = [d for d in df[label_col].unique() if d != healthy_label]

    for feat in features:
        q1_h, q3_h = df.loc[df[label_col]==healthy_label, feat].quantile([0.25, 0.75])
        overlap_maladies = []
        for disease in diseases:
            q1_m, q3_m = df.loc[df[label_col]==disease, feat].quantile([0.25, 0.75])
            # Condition de chevauchement
            if (q1_h < q3_m) and (q3_h > q1_m):
                overlap_maladies.append(disease)
        results.append({
            "feature": feat,
            "overlapping_diseases": overlap_maladies,
            "n_overlap": len(overlap_maladies)
        })
    # On retourne un DataFrame trié par n_overlap décroissant
    return pd.DataFrame(results).sort_values("n_overlap", ascending=False)

# --- Préparation de 'disease_label' 
disease_cols = [c for c in df.columns if c.startswith("disease_")]
mask_d = df[disease_cols].applymap(lambda x: x is True or x == 1)
idx_d  = mask_d.values.argmax(axis=1)
df["disease_label"] = [disease_cols[i].replace("disease_", "") for i in idx_d]

features = [
    'contrast', 'energy', 'homogeneity', 'dissimilarite', 'Correlation',
    'mean_H', 'mean_S', 'mean_V', 'std_R', 'std_G', 'std_B',
    'hu_1', 'hu_2', 'hu_3', 'hu_4', 'hu_5', 'hu_6', 'hu_7'
]

overlap_df = iqr_overlap_table(df, features, label_col="disease_label", healthy_label="healthy")
pd.set_option('display.max_colwidth', 120)  # Pour voir les maladies en entier
print(overlap_df[["feature", "overlapping_diseases", "n_overlap"]])
df.to_excel("Results_chevauchements_IQR.xlsx", index=False)


In [ ]:
# --- Préparation de 'disease_label' 
disease_cols = [c for c in df.columns if c.startswith("disease_")]
mask_d = df[disease_cols].applymap(lambda x: x is True or x == 1)
idx_d  = mask_d.values.argmax(axis=1)
df["disease_label"] = [disease_cols[i].replace("disease_", "") for i in idx_d]

quant_vars_disease = ['contrast', 'energy', 'homogeneity', 'dissimilarite', 'Correlation', 'mean_H', 'mean_S', 'mean_V', 'std_R', 'std_G', 'std_B', 'hu_1', 'hu_2', 'hu_3', 'hu_4', 'hu_5', 'hu_6', 'hu_7']

# Liste des maladies dans l'ordre d'apparition
classes = pd.unique(df["disease_label"].dropna()).tolist()
palette = px.colors.qualitative.Plotly
color_map = {cls: palette[i % len(palette)] for i, cls in enumerate(classes)}

n = len(quant_vars_disease)
n_cols = 2  # Par exemple, 2 colonnes pour éviter une seule colonne trop longue
n_rows = (n + n_cols - 1) // n_cols

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=[v.replace("_", " ") for v in quant_vars_disease],
    vertical_spacing=0.1, horizontal_spacing=0.07
)

for i, feat in enumerate(quant_vars_disease):
    row = i // n_cols + 1
    col = i % n_cols + 1
    for j, cls in enumerate(classes):
        y_vals = df.loc[df["disease_label"] == cls, feat]
        fig.add_trace(
            go.Box(
                y=y_vals,
                name=str(cls),
                marker_color=color_map[cls],
                boxmean='sd',
                showlegend=(i==0),  # une seule légende commune
            ),
            row=row, col=col
        )
    # X et Y titres
    fig.update_yaxes(title_text=feat.replace("_", " "), row=row, col=col)
    fig.update_xaxes(title_text="Maladie", row=row, col=col)

fig.update_layout(
    title="Distribution des variables quantitatives par maladie",
    height=350 * n_rows,
    width=850 * n_cols,
    margin=dict(l=30, r=30, t=80, b=40),
    legend=dict(title="Maladie", orientation="v", x=1.03, y=1)
)

fig.show()
fig.write_html("Boxplots_toutes_var_disease.html", include_plotlyjs='cdn')
print("→ Tous les boxplots sauvegardés dans : Boxplots_toutes_var_disease.html")

In [ ]:
# Colonnes des moyennes des canaux RGB
rgb_means = ['mean_R', 'mean_G', 'mean_B']

# histogrammes superposés avec chaîne de couleurs
plt.figure(figsize=(8, 6))
colors = ['r', 'g', 'b']
for feat, col in zip(rgb_means, colors):
    plt.hist(df[feat].dropna(), bins=30, alpha=0.5, label=feat, color=col)
plt.title("Histograms of mean RGB values")
plt.xlabel("Valeur")
plt.ylabel("Count")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=df, x='Est_Saine', y='netteté')
plt.xticks([0,1], ['Malade','Saine'])
plt.title("Boxplot de la netteté selon l'état de la feuille")
plt.ylabel("Netteté (laplacian var)")
plt.xlabel("")
plt.tight_layout()
plt.show()

In [ ]:
# Affichage des histogrammes des caractéristiques du dataframe
# 1. Colonnes à exclure
exclude = ['ID_Image', 'Est_Saine', 'is_black', 'dimensions']
exclude += [col for col in df.columns if col.startswith(('plant_', 'disease_'))]

# 2. Sélection des features numériques à tracer
features = [
    col for col in df.select_dtypes(include=[np.number]).columns
    if col not in exclude
]

# 3. Définir la taille de la grille de subplots
n_features = len(features)
n_cols = 4
n_rows = int(np.ceil(n_features / n_cols))

# 4. Création des subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 3*n_rows))
axes = axes.flatten()

# 5. Boucle de tracé
for idx, feature in enumerate(features):
    ax = axes[idx]
    ax.hist(df[feature].dropna(), bins=30)
    ax.set_title(feature)
    ax.set_xlabel(feature)
    ax.set_ylabel("Count")

# 6. Désactiver les sous-parcelles non utilisées
for j in range(idx+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Affichage de heatmap des caractéristiques du dataframe 
# 2. Colonnes à exclure (ID, labels, one-hot encodings)
exclude = ['ID_Image', 'Est_Saine', 'is_black', 'dimensions']
exclude += [col for col in df.columns if col.startswith(('plant_', 'disease_'))]

# 3. Sélection des features numériques
numeric_cols = [
    col for col in df.select_dtypes(include=[np.number]).columns
    if col not in exclude
]

# 4. Calcul de la matrice de corrélation
corr_matrix = df[numeric_cols].corr()

# 5. Affichage avec seaborn pour plus de lisibilité
plt.figure(figsize=(14, 12))
sns.heatmap(
    corr_matrix,
#    mask=np.triu(np.ones_like(corr_matrix, dtype=bool)),  # masque la moitié supérieure
    cmap='vlag',                                           # palette divergeante
    annot=False,                                           # mettre True pour afficher les valeurs 
    fmt=".2f",
    cbar_kws={'label': 'Coefficient de corrélation'}
)
plt.title('Heatmap de corrélation des features numériques')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()